<a id='top'></a>

# Watershed lake, Wetland and Glacial cover.

## Glacier Cover
Calculate percent Glacier cover using Randolph Glacial Inventory data. Calculate percent cover using tabulate area.
* **wtd_glacier_per = percent of watershed with glacier cover**

## Wetland cover
Calculate percent wetland cover using NLCD data converted to binary grid as follows:
wetlands = Con(nlcd_ras == 90, 1, Con(nlcd_ras == 95, 1, 0))
Calculate percent cover using tabulate area.
* **wtd_wet_per = percent of watershed with wetland cover**

### Lake Cover
Calculate percent lake cover (ftype = lakes/ponds) NHDPlus data for regions with those data and most current NHD data
 available for regions without. Calculate percent cover using tabulate area.

* **wtd_lake_per = percent of watershed with lake cover**

## Import modules

In [1]:
import arcpy
import os
import datetime
import time
import sys
arcpy.env.overwriteOutput = True
today = datetime.datetime.now()
# Make the time stamp.
time_stamp = '{:%d%m%Y}'.format(today)
print(time_stamp)

path = os.getcwd()
print (path)
print (sys.base_exec_prefix)

14102021
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\landcover
C:\Users\dwmerrigan\AppData\Local\Programs\ArcGIS\Pro\bin\Python\envs\arcgispro-py3


## Collect data and set working output locations
Set data_dir to folder containing all AKSSF regional subfolders/geoadatabases
* RS data folder: data_dir = r"W:\GIS\AKSSF"
* DM data folder: data_dir = r"D:\GIS_Temp\AKSSF_BeckyCopy\AKSSF"

In [2]:
# Set AKSSF Data directory
# data_dir = r"W:\GIS\AKSSF"

# dm local
data_dir = r"D:\GIS_Temp\AKSSF\"

arcpy.env.workspace = data_dir
regions = arcpy.ListWorkspaces(workspace_type="Folder")
rois= []
print (regions)

# Path to create output folder/gdb
temppath = r"D:\GIS_temp" # Output folder
dirname = 'AKSSF_land_met'
tempgdbname = 'AKSSF_land_met.gdb'
temp_dir = os.path.join(temppath, dirname)

# Create temporary working gdb
if not arcpy.Exists(temp_dir):
    os.makedirs(temp_dir)
else:
    print('Working Folder already created...', temp_dir)

outcheck = os.path.join(temp_dir, tempgdbname)

if arcpy.Exists(outcheck):
    print ('Output location already exists', outcheck)
    outgdb = outcheck
if not arcpy.Exists(outcheck):
    print('Creating output GDB')
    tempgdb = arcpy.CreateFileGDB_management(temp_dir,tempgdbname)
    print ('Output geodatabase created at', outcheck)
    outgdb = tempgdb.getOutput(0)

['D:\\GIS_temp\\AKSSF_BeckyCopy\\AKSSF\\Bristol_Bay', 'D:\\GIS_temp\\AKSSF_BeckyCopy\\AKSSF\\Cook_Inlet', 'D:\\GIS_temp\\AKSSF_BeckyCopy\\AKSSF\\Copper_River', 'D:\\GIS_temp\\AKSSF_BeckyCopy\\AKSSF\\Kodiak', 'D:\\GIS_temp\\AKSSF_BeckyCopy\\AKSSF\\Prince_William_Sound']
Working Folder already created... D:\GIS_temp\AKSSF_land_met
Output location already exists D:\GIS_temp\AKSSF_land_met\AKSSF_land_met.gdb


In [3]:
# Set regional workspaces from AKSSF data folder and store in list
arcpy.env.workspace = data_dir
regions  = arcpy.ListWorkspaces(workspace_type="Folder")
regions

['D:\\GIS_temp\\AKSSF_BeckyCopy\\AKSSF\\Bristol_Bay',
 'D:\\GIS_temp\\AKSSF_BeckyCopy\\AKSSF\\Cook_Inlet',
 'D:\\GIS_temp\\AKSSF_BeckyCopy\\AKSSF\\Copper_River',
 'D:\\GIS_temp\\AKSSF_BeckyCopy\\AKSSF\\Kodiak',
 'D:\\GIS_temp\\AKSSF_BeckyCopy\\AKSSF\\Prince_William_Sound']

## Begin GIS portion
- May be faster and save more space to create aspect rasters here using extract by mask?
- May be able to use aspect raster with tabulate intersection to calculate percent North (aspects from 315-45 degrees)

### <b>UPDATE - 2021-10-12 Do not run aspect metrics at this time. Only need to run percent North for watersheds</b>

In [6]:
from arcpy.sa import *
import re

# Set data_dir equal to folder containing AKSSF regional subfolders containing GDBs and raster datasets
arcpy.env.workspace = data_dir
arcpy.env.overwriteOutput = True
regions  = arcpy.ListWorkspaces(workspace_type="Folder")

# Lists to store output tables
wtd_wet_tables = []
wtd_glacier_tables = []
wtd_lake_tables = []

# Set path to datasets common to both networks
nlcd_ras = r''
rgi = r''

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()

# Seperate data by
nhdplus_dat = ['Cook_Inlet','Copper_River']
tauDem_dat = ['Bristol_Bay', 'Kodiak', 'Prince_William_Sound']

# Loop through all processing areas
rois = nhdplus_dat + tauDem_dat
rois = ['Copper_River']
for roi in rois:
# Loop through regional folders
    for region in regions:
        print (region)
        if roi in str(region):
            print(f'{region} found for {roi}')
            if roi in nhdplus_dat:
                # NHDPLus raster data for DM pc, variables will be overwritten below if they exist in source folder.
                wbds = r"D:\Basedata\AKSSF_Basedata\AKSSF_NHDPlus_north_aspect.tif"
                print(f'NHD data: north raster - {nor_rast}')
                print('----------')

            elif roi in tauDem_dat:
                # TauDEM raster data for DM pc, variables will be overwritten below if they exist in source folder.
                nor_rast = r"D:\Basedata\AKSSF_Basedata\AKSSF_Composite_10m_north_aspect.tif"
                asp_rast = r"D:\Basedata\AKSSF_Basedata\AKSSF_Composite_10m_aspect.tif"
                elev_rast = r"D:\Basedata\AKSSF_Basedata\AKSSF_Composite_10m_extract.tif"
                print(f'TauDem data: north raster - {nor_rast}')
                print(f'         aspect raster - {asp_rast}')
                print(f'      elevation raster - {elev_rast}')
                print('----------')

            stats_fields = ['MEAN','MAX','MIN','STD']
            wtds = []
            cats = []
            # Start iter timing function
            iteration_start = time.time()
            # Set workspace to region folder
            arcpy.env.workspace = region
            gdb = arcpy.ListWorkspaces(workspace_type='FileGDB')
            walk = arcpy.da.Walk(region, datatype = ['FeatureClass','RasterDataset'])
            for dirpath, dirnames, filenames in walk:
                for filename in filenames:
                    # Create list of watersheds
                    if (("wtd" in filename) and ("merge" not in filename)):
                        wtds.append(os.path.join(dirpath, filename))

                    # Select elevation raster
                    elif 'elev_merge' in filename:
                        elev_rast = os.path.join(dirpath, filename)

                    # # Select aspect raster
                    # elif 'aspect' in filename:
                    #     asp_rast = os.path.join(dirpath, filename)

                    # Select north raster
                    elif 'north_asp' in filename:
                        nor_rast = os.path.join(dirpath, filename)

                    # Select catch_int fc (catchments of interest for region) and make a copy
                    elif 'cats_intersect' in filename:
                        cats = arcpy.FeatureClassToFeatureClass_conversion(os.path.join(dirpath,filename),
                                                                           tempgdb,'cat_copy')
                        arcpy.AddField_management(cats,"NHDPlusID_txt",field_type='TEXT')
                        # Copy NHDPlusID as text field for use in zonal stats
                        with arcpy.da.UpdateCursor(cats, ['NHDPlusID','NHDPlusID_txt']) as cur:
                            for row in cur:
                                row[1] = roi + "_" + str(row[0])
                                # Update rows
                                cur.updateRow(row)
                            del(row)
                        del(cur)

            print (f'Calculating aspect metrics and percent north for catchments & watersheds of interest in {roi}'
                   f' region')
            print ('----------')
            print(f'Geodatabase: {gdb}')
            print ('----------')
            print (f'Elevation Raster: {elev_rast}')
            print ('----------')
            print (f'North Aspect Raster: {nor_rast}')
            print ('----------')
            print (f'{len(wtds)} Watersheds to process')
            print ('----------')
            print (f'Catchment intersect {cats} selected')
            print ('----------')

            try:
                mergestart = time.time()
                # Set output names/paths for watershed and catchment tables
                wtd_merge_name = roi + "_Watersheds_Merge"
                wtd_merge_path = os.path.join(outgdb,wtd_merge_name)

                # # Aspect variables
                # wtd_merge_asp_table_name = roi + "_Watersheds_Merge_AspectZstats"
                # wtd_merge_asp_table_path = os.path.join(outgdb, wtd_merge_asp_table_name)
                # cat_asp_table_name = roi + "_Catchments_AspectZstats"
                # cat_asp_table_path = os.path.join(outgdb, cat_asp_table_name)

                # Percent North variables
                wtd_merge_pernorth_table_name = roi + "_Watersheds_Merge_PercentNorth"
                wtd_merge_pernorth_table_path = os.path.join(outgdb, wtd_merge_pernorth_table_name)
                # cat_pernorth_table_name = roi + "_Catchments_PercentNorth"
                # cat_pernorth_table_path = os.path.join(outgdb, cat_pernorth_table_name)

                # Elevation variables
                wtd_merge_elev_table_name = roi + "_Watersheds_Merge_ElevZstats"
                wtd_merge_elev_table_path = os.path.join(outgdb, wtd_merge_elev_table_name)
                cat_elev_table_name = roi + "_Catchments_ElevZstats"
                cat_elev_table_path = os.path.join(outgdb, cat_elev_table_name)

                # Merge watersheds if they are not already created
                if not arcpy.Exists(wtd_merge):
                    wtd_merge = arcpy.Merge_management( wtds, wtd_merge_path ,add_source='ADD_SOURCE_INFO')
                    # Add wtd_id field
                    arcpy.AddField_management(wtd_merge,'wtd_id',field_type='TEXT')
                    # Add region field
                    arcpy.AddField_management(wtd_merge,'region',field_type='TEXT')
                    # Populate watershed id and region information using update cursor - faster than field calc
                    with arcpy.da.UpdateCursor(wtd_merge,['MERGE_SRC','wtd_id','region']) as cur:
                        for row in cur:
                            row[1] = re.findall('\d+', row[0])[0]
                            row[2] = roi
                            # Update
                            cur.updateRow(row)
                        del(row)
                    del(cur)
                    mergestop = time.time()
                    mergetime = int (mergestop - mergestart)
                    print(f'Watershed Merge for {roi} Elapsed time: ({datetime.timedelta(seconds=mergetime)})')
                    print('----------')

                # Begin Zonal Stats
                zstat_start = time.time()
                print(f'Begin zonal statistics min/mean/max std dev for watersheds and catchments in {roi} region')

                # Elevation Zonal statistics  for watersheds
                wtd_elev_metrics_table = ZonalStatisticsAsTable(in_zone_data = wtd_merge,
                                                                zone_field = "wtd_id",
                                                                in_value_raster = elev_rast,
                                                                out_table = wtd_merge_elev_table_path,
                                                                statistics_type='ALL'
                                                                )
                # Add region identifier field for watershed tables                                                )
                arcpy.AddField_management(wtd_elev_metrics_table,'region',field_type='TEXT')
                # Update region field
                with arcpy.da.UpdateCursor(wtd_elev_metrics_table,'region') as cur:
                    for row in cur:
                        row[0] = roi
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # Append watershed elev table to list
                wtd_elev_ztables.append(wtd_elev_metrics_table)

                # Elevation zonal statistics for catchments
                cat_elev_metrics_table = ZonalStatisticsAsTable(in_zone_data = cats ,
                                                                zone_field = "NHDPlusID_txt",
                                                                in_value_raster = elev_rast,
                                                                out_table = cat_elev_table_path,
                                                                statistics_type='ALL'
                                                                )
                # Add region identifier field for catchment table
                arcpy.AddField_management(cat_elev_metrics_table,'region',field_type='TEXT')

                # Update region field
                with arcpy.da.UpdateCursor(cat_elev_metrics_table,'region') as cur:
                    for row in cur:
                        row[0] = roi
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # Append catchment elev table to list
                cat_elev_ztables.append(cat_elev_metrics_table)

                # # Aspect Zonal statistics  for watersheds
                # wtd_asp_metrics_table = ZonalStatisticsAsTable(in_zone_data = wtd_merge, zone_field ="wtd_id",
                #                                                in_value_raster = asp_rast, out_table = wtd_merge_asp_table_path,
                #                                                statistics_type='ALL')
                # arcpy.AddField_management(wtd_asp_metrics_table, 'region', field_type='TEXT')
                # arcpy.CalculateField_management(wtd_asp_metrics_table, 'region', 'roi')
                # wtd_asp_ztables.append(wtd_asp_metrics_table)

                # # Aspect Zonal statistics for catchments
                # cat_asp_metrics_table = ZonalStatisticsAsTable(in_zone_data = cats, zone_field ="NHDPlusID_txt",
                #                                                in_value_raster = asp_rast, out_table = cat_asp_table_path,
                #                                                statistics_type='ALL')
                # cat_asp_ztables.append(cat_asp_metrics_table)

                zstat_stop = time.time()
                zstat_time = int (zstat_stop - zstat_start)
                print(f'Zonal Stats for {roi} Elapsed time: ({datetime.timedelta(seconds=zstat_time)})')
                print('----------')

                # Tabulate Area with north grid and catchments/watersheds
                tabarea_start = time.time()
                print(f'Begin tabulate area of north facing cells for watersheds and catchments in {roi} region')
                # Percent North Tabulate area for watersheds
                wtd_per_north_tabarea = arcpy.sa.TabulateArea(in_zone_data= wtd_merge,
                                                              zone_field='wtd_id',
                                                              in_class_data=nor_rast,
                                                              class_field="Value",
                                                              out_table=wtd_merge_pernorth_table_path
                                                              )
                # Add region and percent north fields
                arcpy.AddField_management(wtd_per_north_tabarea, 'region', field_type='TEXT')
                arcpy.AddField_management(wtd_per_north_tabarea, 'north_wtd', field_type='Float')

                with arcpy.da.UpdateCursor(wtd_per_north_tabarea,['wtd_id','region', 'VALUE_0',
                                                                        'VALUE_1', 'north_wtd']) as cur:
                    for row in cur:
                        row[1] = roi
                        row[4] = row[3]/(row[3]+row[2])*100
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # Append watershed percent north table to list
                wtd_pernorth_taba_tables.append(wtd_per_north_tabarea)

                # # Percent North Tabulate Area for catchments
                # cat_per_north_tabarea = arcpy.sa.TabulateArea(in_zone_data= cats, zone_field='NHDPlusID',
                #                                             in_class_data=nor_rast,"Value",
                #                                             out_table=cat_pernorth_table_path)

                # # Add and calculate region identifier field for catchment table
                # arcpy.AddField_management(cat_per_north_tabarea, 'wtd_id', field_type='Float')
                # arcpy.AddField_management(cat_per_north_tabarea, 'region', field_type='TEXT')
                # arcpy.AddField_management(cat_per_north_tabarea, 'north_cat', field_type='Float')
                #
                # with arcpy.da.UpdateCursor(cat_per_north_tabarea,['NHDPlusID','region', 'VALUE_0',
                #                                                         'VALUE_1', 'north_cat']) as cur:
                #     for row in cur:
                #         row[1] = roi
                #         row[4] = row[3]/(row[3]+row[2])*100
                #         # Update
                #         cur.updateRow(row)
                #     del(row)
                # del(cur)
                # cat_pernorth_taba_tables.append(cat_per_north_tabarea)
                tabarea_stop = time.time()
                tabarea_time = int (tabarea_stop - tabarea_start)
                print(f'Tabulate area for {roi} Elapsed time: ({datetime.timedelta(seconds=tabarea_time)})')
                print('----------')

            except:
                e = sys.exc_info()[1]
                print(e.args[0])
                arcpy.AddError(e.args[0])

    else:
        print(f'Region {str(roi)} not found in {region}')
# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print('----------')



D:\GIS_temp\AKSSF_BeckyCopy\AKSSF\Bristol_Bay
D:\GIS_temp\AKSSF_BeckyCopy\AKSSF\Cook_Inlet
D:\GIS_temp\AKSSF_BeckyCopy\AKSSF\Copper_River
D:\GIS_temp\AKSSF_BeckyCopy\AKSSF\Copper_River found for Copper_River
NHD data: north raster - D:\Basedata\AKSSF_Basedata\AKSSF_NHDPlus_north_aspect.tif
         aspect raster - D:\Basedata\AKSSF_Basedata\AKSSF_NHDPlus_aspect.tif
      elevation raster - D:\Basedata\AKSSF_Basedata\AKSSF_NHDPlus_elev_cm.tif
----------
Calculating aspect metrics and percent north for catchments & watersheds of interest in Copper_River region
----------
Geodatabase: ['D:\\GIS_temp\\AKSSF_BeckyCopy\\AKSSF\\Copper_River\\Copper_River.gdb']
----------
Elevation Raster: elev_merge
----------
28 Watersheds to process
----------
Catchment intersect memory\cat_copy selected
----------
Watershed Merge for Copper_River Elapsed time: (0:00:04)
----------
Begin zonal statistics min/mean/max std dev for watersheds and catchments in Copper_River region
Zonal Stats for Copper_River E

## Drop unnecessary fields and rename as needed from merged tables.
- Create Key value dictionary and use update cursor to rename fields.

In [ ]:
# Merge tables together
        # wtd_per_north = arcpy.Merge_management(wtd_elev_ztables, wtd_per_north_table_out)
        # cat_per_north = arcpy.Merge_management(cat_asp_ztables, cat_per_north_table_out)
        # Use update cursor to calculate percent North
                        # with arcpy.da.UpdateCursor(wtd_per_north,['VALUE_0','VALUE_1','wtd_aspect_mn']) as cur:
                        #     for row in cur:
                        #         row[2] = row[1]/(row[0] + row[1]) *100
                        #         cur.updateRow(row)
                        #     del(row)
                        # del(cur)

# Select tables and rename fields

# Export to csv
